### We will pre process the data first
### Steps involved in data preprocessing
#### 1. Take the convo lines and convo ids which has the list of lines                         Each convo id is a conversation in the scene which has multiple lines. These           lines are list of line id (like foreign key to convo lines file).
#### 2. Take of the delimeter from both files and take only Convo id, line id and line         text.
#### 3. Based on the Covo Id list of lines  and line ID line text, we will create two           files . One file is questions: Which has all the lines. Another file is answers         which has all the lines except first line(Where first line is a question). This         answer woould be sent as input or as hidden state while training.
#### 4. We will do some data cleaning, where we will replace short words into actual           words. eg: replace wern't to were not. wouldn't to would not etc. and take of           the special characters like +,-,*,/,-,),(etc., except ! and ? where these are           used to detect the sentiment analysis.
#### 5. Take the count of the words in both questions and answers (Usually both are             same. Need to find the purpose).
#### 6. Tokenization and non-frequent words: these 2 are the important concepts                 We will remove non-frequent words from the wordcount dict. We have a threshold         for the count(eg: words less than 20 occurances can  be discarded), WE will             create  questionwordcount and answerwordcount dicts from this.                         Tokenization: 

In [1]:
import numpy as np
import tensorflow as tf
import time
import re

#### Now we will read movie lines and conversations

In [2]:
lines=open('movie_lines.txt', encoding='utf-8', errors='ignore').read()
lines=lines.split('\n')

In [3]:
type(lines)

list

In [4]:
conversation=open('movie_conversations.txt', encoding='utf-8', errors='ignore').read()
conversation=conversation.split('\n')

### Lines have line Id and text in the end

In [5]:
print(lines[45])

L281 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Where?


### Conversation has the list of lines which represents a conversation
### These conversation has Ids and lis of lines

In [6]:
print(conversation[34])

u0 +++$+++ u4 +++$+++ m0 +++$+++ ['L589', 'L590', 'L591']


## Now we will create a dictionary which matches Lines and Conversation Id

In [7]:
id2line={}

In [8]:
for line in lines:
    _line = line.split(' +++$+++ ')
    # Each line has 5 elements we will make sure we have 5 elements in the list.
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [9]:
print(id2line['L1045'])

They do not!


### We got the lines and now we will get the conversations into list

In [10]:
conversations_id=[]

## The last row of conversation is empty s owe discard it.

In [11]:
for details in conversation[:-1]:
    #WE get the last part, so we will split the data , FYI the last part is a list of 
    # lines
    _conversation_list = details.split(' +++$+++ ')[-1]
    # We got the list of lines. But there might be rows which has some spaces and 
    ## the value in the lat row is not a list but a string which has a square bracket
    ## WE have to remove that.
    remove_bracs=_conversation_list[1:-1]
    remove_single_quotes=remove_bracs.replace("'","")
    remove_unwanted_space=remove_single_quotes.replace(" ","")
    conversations_id.append(remove_unwanted_space.split(','))

In [12]:
conversations_id[:10]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366']]

## We now have cleaned and mapped the convos and lines from the text
## Next job is to find the questions and the answers 
### This can be done by following examples:
#### 1. Any conversation which start would be considered as question and second would be answer
#### 2. So, here the first in the list of lines would be the quetion and second would be the answer.
#### 3. And that answer will be passed as a question in the second row and for that question would be in the third row and so on..
#### 3. Now we will seggregate the questions and answers and store the text from id2line dict

In [13]:
questions=[]
answers=[]

In [14]:
for line_list in conversations_id:
    for line_no in range(len(line_list)-1):
        #print('hello')
        questions.append(id2line[line_list[line_no]])
        answers.append(id2line[line_list[line_no+1]])
    

In [15]:
questions[:10]

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "You're asking me out.  That's so cute. What's your name again?",
 "No, no, it's my fault -- we didn't have a proper introduction ---",
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Why?',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 'Gosh, if only we could find Kat a boyfriend...']

In [16]:
answers[:10]

["Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?",
 'Forget it.',
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Seems like she could get a date easy enough...',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 "That's a shame.",
 'Let me see what I can do.']

### WE have got the questions and answers.
### Now time to clean some data
#### We might have some text like i'm, we're etc.,
#### WE will replace that with the actual text like i am, We are etc.,
#### For that we will create a function and replace the words like

In [17]:
def clean_text(text):
    # We will utilize re library to replace
    # WE will convert the text to lower case
    text=text.lower()
    
    text = re.sub(r"i'm", "i am",text)
    text= re.sub(r"he's","he is",text)
    text=re.sub(r"she's", "she is",text)
    text=re.sub(r"that's","that is",text)
    text=re.sub(r"what's","what is",text)
    text=re.sub(r"where's","where is",text)
    text=re.sub(r"\'ll"," will",text)
    text=re.sub(r"\'ve"," have",text)
    text=re.sub(r"\'d'"," would",text)
    text=re.sub(r"\'re"," are",text)
    text=re.sub(r"won't","will not",text)
    text=re.sub(r"can't","cannpt",text)
    text=re.sub(r"[-()\"#/@;:<>{}+=~|.?,]"," ",text)
    #print(text)
    return text
    

### We will apply function to both question and ansewers

In [18]:
clean_question=[]
for q_list in questions:
        clean_question.append(clean_text(q_list))

In [19]:
clean_answers=[]
for a_list in answers:
    clean_answers.append(clean_text(a_list))

In [20]:
clean_question[:10]

['can we make this quick   roxanne korrine and andrew barrett are having an incredibly horrendous public break  up on the quad   again ',
 "well  i thought we'd start with pronunciation  if that is okay with you ",
 'not the hacking and gagging and spitting part   please ',
 'you are asking me out   that is so cute  what is your name again ',
 "no  no  it's my fault    we didn't have a proper introduction    ",
 'cameron ',
 'the thing is  cameron    i am at the mercy of a particularly hideous breed of loser   my sister   i cannpt date until she does ',
 'why ',
 'unsolved mystery   she used to be really popular when she started high school  then it was just like she got sick of it or something ',
 'gosh  if only we could find kat a boyfriend   ']

In [21]:
clean_answers[:20]

["well  i thought we'd start with pronunciation  if that is okay with you ",
 'not the hacking and gagging and spitting part   please ',
 "okay    then how 'bout we try out some french cuisine   saturday   night ",
 'forget it ',
 'cameron ',
 'the thing is  cameron    i am at the mercy of a particularly hideous breed of loser   my sister   i cannpt date until she does ',
 'seems like she could get a date easy enough   ',
 'unsolved mystery   she used to be really popular when she started high school  then it was just like she got sick of it or something ',
 'that is a shame ',
 'let me see what i can do ',
 'right   see   you are ready for the quiz ',
 "i don't want to know how to say that though   i want to know useful things  like where the good stores are   how much does champagne cost   stuff like chat   i have never in my life had to point out my head to someone ",
 "that is because it's such a nice one ",
 'forget french ',
 "well  there's someone i think might be   ",
 'where '

### We have cleaned the data of questions and answers.
#### Now we have to create a dictionary which maps each word to its number of occurences.
#### It will map like each word to how many times it has occured.

In [22]:
word2count={}

In [23]:
### Lets start with questions
for ques in clean_question:
    # Now go through all the words in each question
    for word in ques.split():
        if word not in word2count:
            word2count[word]=1
        else:
            word2count[word] += 1

In [24]:
### Now with answers
for ans in clean_answers:
    for word in ans.split():
        if word not in word2count:
            word2count[word]=1
        else:
            word2count[word] += 1

### now we have the number of occurences
## now lets dice into two important concepts
### Tokenization and filtering the non-frequent words

### in this we will have twp dicts which map unique words from questions to one integers and another dict will map the answer unique words to another integer

### First we will go through non-frequent words

In [25]:
threshold = 20
queswords2int={}
answords2int={}

In [26]:
### Loop the wordcount and ad them in question dict
word_number=0
for word,count in word2count.items():
    if count > threshold:
        # We will assingn that word to unique number
        queswords2int[word] = word_number
        word_number += 1
    

In [27]:
### Loop the wordcount and add it in answerdict
word_number=0
for word,count in word2count.items():
    if count > threshold:
        answords2int[word]=word_number
        word_number+=1

In [28]:
answords2int['can']

0

### Now we will Add unique tokens to the dicts
#### This tokens are used in encoding and decoding in seq2seq model.

In [29]:
tokens=['<PAD>', '<EOS>','<OUT>','<SOS>']
## Lets append these values to the word count
for token in tokens:
    queswords2int[token]=len(queswords2int)+1
    answords2int[token]=len(answords2int)+1

In [30]:
queswords2int['<EOS>']

8592

### Now we have questions words dict with words as keys and count as values.
#### for answers words we will swap those like count wouldbe the key and word would be the value

In [31]:
answords2word={c:w for w,c in answords2int.items() }

In [32]:
answords2word[3454]

'concept'

### Now we are going to add the EOS string to every answer.

In [33]:
for i in range(len(clean_answers)):
    clean_answers[i]= clean_answers[i] + ' <EOS>'
    
    

In [34]:
clean_answers[345]

"yeah  well  don't let it get out <EOS>"

### Now we will translate all the questions into integers
### WE will loop all the words from clean question and map with words2int. if any word form question doesnot found in quesiton then we will replace it with out index number, otherwise normal index number of what that word have in the question2int dict

In [35]:
questions2int=[]

In [36]:
###

In [37]:
for ques in clean_question:
    ints=[]
    for word in ques.split():
        if word not in queswords2int:
            ints.append(queswords2int['<OUT>'])
        else:
            ints.append(queswords2int[word])
    questions2int.append(ints)

In [38]:
questions2int[:5]

[[0,
  1,
  2,
  3,
  4,
  8593,
  8593,
  5,
  6,
  8593,
  7,
  8,
  9,
  10,
  8593,
  11,
  12,
  13,
  14,
  15,
  8593,
  16],
 [17, 18, 19, 20, 21, 22, 8593, 23, 24, 25, 26, 22, 27],
 [28, 15, 8593, 5, 8593, 5, 8593, 29, 30],
 [27, 7, 31, 32, 33, 24, 25, 34, 35, 36, 25, 37, 38, 16],
 [39, 39, 40, 41, 42, 1, 43, 44, 45, 46, 8593]]

#### WE will conver the same for answers


In [39]:
answer2int=[]
for ans in clean_answers:
    ints=[]
    for word in ans.split():
        if word not in answords2int:
            ints.append(answords2int['<OUT>'])
        else:
            ints.append(answords2int[word])
    answer2int.append(ints)

In [40]:
answords2int['hi']

149

In [41]:
answords2word[4]

'quick'

In [42]:
answer2int[:5]

[[17, 18, 19, 20, 21, 22, 8593, 23, 24, 25, 26, 22, 27, 8592],
 [28, 15, 8593, 5, 8593, 5, 8593, 29, 30, 8592],
 [26, 73, 97, 1558, 1, 888, 33, 502, 402, 8593, 216, 247, 8592],
 [250, 74, 8592],
 [47, 8592]]

### now we will sort questions and answers 
### This we will do because when learning when we preset large sentence then the model could not able lo learn
### Here we will restrict the question with 25 words and not more than that (This is variable where we can change)

In [43]:
sorted_questions=[]
sorted_answers=[]

In [44]:
for length in range(1,26):
    for index,words in enumerate(questions2int):
        if len(words) == length:
            sorted_questions.append(questions2int[index])
            
            sorted_answers.append(answer2int[index])

In [45]:
questions2int[1]

[17, 18, 19, 20, 21, 22, 8593, 23, 24, 25, 26, 22, 27]

In [50]:
sorted_questions[0]

[47]

In [51]:
sorted_answers[0]

[15,
 48,
 25,
 47,
 18,
 49,
 50,
 15,
 51,
 52,
 45,
 53,
 8593,
 54,
 52,
 55,
 41,
 56,
 18,
 57,
 58,
 59,
 60,
 61,
 8592]